In [1]:
import numpy as np
import pandas as pd
import plotly.io as pio
import vectorbtpro as vbt

from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.data.labeling import label_data_extrema_multi
from vctr.data.lstm_preprocessor import preprocess_data_2d
from vctr.features.feature_engineering import add_features
from vctr.models.rfc_multi import RFClassifier

In [2]:
symbols = ['ETH', 'BTC', 'ADA']
timeframe = '1h'
freq = timeframe.replace('m', 't')

data = [
    get_data_with_features_and_labels(symbol, timeframe=timeframe, label_args=(0.06, 0.005), separate=False) for symbol in symbols
]


In [3]:
df = pd.concat(data)

X = df.drop(['label'], axis=1)
y = df['label']

print(f'Samples before: {len(df)}')

X_train, y_train, X_test, y_test = preprocess_data_2d(X, y, 0.3)
X_test_backup = X_test.copy()

# for k in ['open', 'high', 'low', 'close', 'volume']:
#     if k in X_train.columns:
#         X_train.drop(k, axis=1, inplace=True)
#     if k in X_test.columns:
#         X_test.drop(k, axis=1, inplace=True)

print(f'Samples after: {len(X_train) + len(X_test)}')


Samples before: 153626
Samples after: 153626


In [4]:
rfc = RFClassifier(3)

In [5]:
y_pred = rfc.train_and_evaluate(X_train, y_train, X_test, y_test)

length: 107539
Accuracy:  58%
F1 (Macro):  31%
Precision (Macro):  37%
Recall (Macro):  37%
Confusion Matrix: 
 [[59985 20303 24609]
 [   53  1246     0]
 [   89     1  1253]]


In [ ]:
X_test.drop(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)

In [6]:
pd.Series(y_pred).value_counts(normalize=True)

0    0.559118
2    0.240489
1    0.200392
dtype: float64

In [7]:
rfc.get_feature_importance(X)

In [20]:
data = X.join(y)
# X_test_backup['close']
close = data['close']

print(len(y_pred))

# y_pred = pd.Series(y_pred)
# pf = vbt.Portfolio.from_signals(
#     data.loc[data.index, 'close'],
#     y_pred == 1,
#     y_pred == 2,
#     freq=freq,
#     init_cash=1,
# )
# print(pf.stats())




38360


In [ ]:
rfc.do_rfe(X_train, y_train, n_features=7)
